In [31]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.read_csv("application_train.csv")

# Detect categorical columns BEFORE coercion
categorical_cols = df.select_dtypes(include=["object"]).columns

# Convert numeric-like columns to numeric
numeric_like_cols = df.columns.difference(categorical_cols)
df[numeric_like_cols] = df[numeric_like_cols].apply(
    lambda col: pd.to_numeric(col, errors='coerce')
)

# Detect numeric columns AFTER coercion
numeric_cols = df.select_dtypes(include=[np.number]).columns


In [32]:
num_imputer = SimpleImputer(strategy="median")
df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])

cat_imputer = SimpleImputer(strategy="most_frequent")
df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])

print("Remaining NaNs:", df.isna().sum().sum())


Remaining NaNs: 0


In [34]:
# Only numeric columns for correlation
numeric_df = df.select_dtypes(include=[np.number])

correlations = numeric_df.corr()['TARGET']
low_corr_cols = correlations[abs(correlations) < 0.01].index.tolist()

df = df.drop(columns=low_corr_cols)



In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = df.drop("TARGET", axis=1)
y = df["TARGET"]

model1 = LogisticRegression(max_iter=10000)
model2 = svm.SVC()
model3 = LinearDiscriminantAnalysis()

model1.fit(X, y)
model2.fit(X, y)
model3.fit(X, y)


from sklearn.metrics import accuracy_score, precision_score, recall_score

# Predictions
pred1 = model1.predict(X)
pred2 = model2.predict(X)
pred3 = model3.predict(X)

print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y, pred1))
print("Precision:", precision_score(y, pred1))
print("Recall:", recall_score(y, pred1))

print("\n=== SVM ===")
print("Accuracy:", accuracy_score(y, pred2))
print("Precision:", precision_score(y, pred2))
print("Recall:", recall_score(y, pred2))

print("\n=== LDA ===")
print("Accuracy:", accuracy_score(y, pred3))
print("Precision:", precision_score(y, pred3))
print("Recall:", recall_score(y, pred3))


ValueError: could not convert string to float: 'Cash loans'

In [ ]:
# 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'LIVINGAPARTMENTS_MEDI', NONLIVINGAPARTMENTS_MEDI'
#not sure if we should drop these? possible proxies for age??


In [ ]:
#
df.isna().sum().sort_values(ascending=False).head(20)


In [ ]:
###MODELS
#logistic regression, possibly with penalization 							
# support vector machines 						
# linear discriminant

from sklearn.linear_model import LogisticRegression as logistic_regression
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as discriminant_analysis
x = df_correlation.drop('TARGET', axis=1)
y = df_correlation['TARGET']
model1 = logistic_regression(max_iter=10000)
model2 = svm.SVC()
model3 = discriminant_analysis()
model1.fit(x, y)
model2.fit(x, y)
model3.fit(x, y)
pred1 = model1.predict(x)
pred2 = model2.predict(x)
pred3 = model3.predict(x)

from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Logistic Regression Accuracy:", accuracy_score(y, pred1))
print("SVM Accuracy:", accuracy_score(y, pred2))
print("LDA Accuracy:", accuracy_score(y, pred3))

print("Logistic Regression Precision:", precision_score(y, pred1))
print("SVM Precision:", precision_score(y, pred2))
print("LDA Precision:", precision_score(y, pred3))

print("Logistic Regression Recall:", recall_score(y, pred1))
print("SVM Recall:", recall_score(y, pred2))
print("LDA Recall:", recall_score(y, pred3))

